In [1]:
import sys, os
sys.path.insert(1, os.getcwd() + '\..\src')

In [2]:
from session_generator import *
from pev_digital_twin import *

# Main Simulation

In [3]:
# List with number of sessions for different days ([number of sessions on day 1, ..., number of sessions on the last day])
daily_sessions = [10]*10

input_gen = InputGen(daily_sessions=daily_sessions, data_file='data/Sessions2.csv', rnd_seeds=(4,5,30))
# Remove outliers:
input_gen.data = input_gen.data[(input_gen.data['DurationHrs'] < 15) & (input_gen.data['DurationHrs'] > 1/6)]
input_gen.data = input_gen.data[input_gen.data['cumEnergy_Wh'] / input_gen.data['DurationHrs'] <= 6700]

input_gen.arrival_gen()
input_gen.duration_gen()
input_gen.energy_gen()

input_df = input_gen.df

#input_df['arrivalMin'] = input_df['arrivalMin'].apply(lambda x: round(x/30)*30 + 30)
#input_df['arrivalMinGlobal'] = input_df['arrivalMinGlobal'].apply(lambda x: round(x/30)*30 + 30)
input_df['durationMin'] = input_df['durationMin'].apply(lambda x: round(x/30)*30 + 90)
input_df['cumEnergy_Wh'] = input_df['cumEnergy_Wh'].apply(lambda x: round(x/10)*10 + 10)

input_df.head(10)

,arrivalDay,arrivalMin,arrivalMinGlobal,durationMin,cumEnergy_Wh
0,0,364,364,180,20
1,0,467,467,630,40
2,0,480,480,270,30
3,0,535,535,630,20
4,0,576,576,450,50
5,0,631,631,510,30
6,0,642,642,570,60
7,0,977,977,300,20
8,0,1016,1016,210,20
9,0,1045,1045,180,10


In [4]:
import simpy

In [5]:
# SIMULATION PARAMETERS
NUM_DAYS = len(daily_sessions)
SIM_RUN_TIME = 1440*NUM_DAYS
CAR_ARR_TIME = 120
CAR_STAY_TIME = 300

env = simpy.Environment()

env.process(first_process(env, input_df, SIM_RUN_TIME))   
env.run(SIM_RUN_TIME + 10)

curr_time =  364
departure_time =  544
requested_energy =  20
curr_time =  467
departure_time =  1097
requested_energy =  40
curr_time =  480
departure_time =  750
requested_energy =  30
curr_time =  535
departure_time =  1165
requested_energy =  20
curr_time =  576
departure_time =  1026
requested_energy =  50
curr_time =  631
departure_time =  1141
requested_energy =  30
curr_time =  642
departure_time =  1212
requested_energy =  60
curr_time =  977
departure_time =  1277
requested_energy =  20


TypeError: 'NoneType' object is not subscriptable

# Output Table

Generate dataframe

In [6]:
df = pd.DataFrame({'session': session, 'choice': user_choice, 'arrival time': arrival_time, 'departure time': departure_time})
df['durationHrs'] = (df['departure time'] - df['arrival time']) / 60
df['arrival time'] = pd.Timestamp(2022, 4, 4, 9) + pd.TimedeltaIndex(df['arrival time'], unit='m')
df['departure time'] = pd.Timestamp(2022, 4, 4, 9) + pd.TimedeltaIndex(df['departure time'], unit='m')

ValueError: All arrays must be of the same length

In [7]:
df['vehicle_model'] = car_type

df['energy_requested'] = energyreq

df['rate_scheduled'] = rate_flex
df['rate_regular'] = rate_asap

df['sch_centsPerHr'] = ((df['rate_scheduled']*100)/df['durationHrs']).round(decimals=0)
df['reg_centsPerHr'] = ((df['rate_regular']*100)/df['durationHrs']).round(decimals=0)

df['energyReq_Wh'] = df['energy_requested']*1000

df = df.drop('rate_scheduled', 1)
df = df.drop('rate_regular', 1)
df = df.drop('energy_requested', 1)

df.head()

NameError: name 'df' is not defined

In [8]:
def hr_func(ts):
    return ts.hour
df['arrival_hour'] = df['arrival time'].apply(hr_func)
df['departure_hour'] = df['departure time'].apply(hr_func)

NameError: name 'df' is not defined

In [9]:
df_hist = pd.DataFrame(df['arrival_hour'])
df_hist['departure_hour']=df['departure_hour']

# plot melted dataframe in a single command
p=sns.histplot(df_hist.melt(), x='value', hue='variable',
             multiple='dodge', shrink=.75, bins=20)

p.set_xlabel("hour", fontsize = 10)
p.set_ylabel("frequency", fontsize = 10);

NameError: name 'df' is not defined

In [10]:
df['vehicle_model'].value_counts().plot(kind='barh', figsize=(10, 10));

NameError: name 'df' is not defined

In [11]:
df = df[df.choice != 'OCCUPIED']

NameError: name 'df' is not defined

View consolidated simulation dataframe

In [ ]:
df.head()

Arrival and departure frequency plot

In [ ]:
df_hist = pd.DataFrame(df['arrival_hour'])
df_hist['departure_hour']=df['departure_hour']

# plot melted dataframe in a single command
p=sns.histplot(df_hist.melt(), x='value', hue='variable',
             multiple='dodge', shrink=.75, bins=20)

p.set_xlabel("hour", fontsize = 10)
p.set_ylabel("frequency", fontsize = 10);

Plot of vehicle types

In [ ]:
df['vehicle_model'].value_counts().plot(kind='barh', figsize=(10, 10));

In [ ]:
df['choice'].value_counts()

Distribution of user choice

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'REGULAR', 'SCHEDULED', 'LEAVE'
sizes = [64, 22, 8]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

Plot session times

In [ ]:
sns.boxplot(x= df['durationHrs']) 
plt.title("Distribution of session times");

Plot energy Requirements

In [ ]:
sns.boxplot(x= df['energyReq_Wh']) 
plt.title("Distribution of Energy Required");